In [1]:
%pip install rockpool

Note: you may need to restart the kernel to use updated packages.


# Import delle librerie

In [2]:
import numpy as np
import torch
import torch.nn as nn

from snntorch import spikeplot as splt
from rockpool.nn.modules import LinearTorch, LIFTorch
from rockpool.nn.combinators import Sequential
from rockpool.parameters import Constant

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt

# Download del datasets

In [3]:
# dataloader arguments
data_root='../data'
# Device and data repository
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Trasformazioni da applicare al Datasets

In [4]:
# Define a transform
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.,), (1,))
])

In [5]:
# Load Datasets
fmnist_train = datasets.FashionMNIST(data_root, train=True, download=True, transform=transform)
fmnist_test = datasets.FashionMNIST(data_root, train=False, download=True, transform=transform)

# Definizione della dimensione del batch

In [6]:
batch_size = 512

# Creazione dei dataloader

In [7]:
# Create DataLoaders
train_loader = DataLoader(fmnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(fmnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [8]:
# Utilities
def plot_mem_rec(mem_rec, batch_size, targets):
    num_steps = len(mem_rec)
    for i in range(0,batch_size):
        fig = plt.figure()
        ax = fig.subplots()
        ax.set_xlim((-10,210))
        ax.set_ylim((-2,2))
        ss = mem_rec[:,i,:]
        plt.plot(range(0,num_steps), ss.cpu().detach())

        plt.title("Output Layer Membrane Output - {}".format(targets[i]))
        plt.xlabel("Time step")
        plt.ylabel("Neuron Number")
        fig.tight_layout()
    plt.show()


def plot_spk_rec(spk_rec, batch_size, targets):
    for i in range(0,batch_size):
        fig = plt.figure()
        ax = fig.subplots()
        ax.set_xlim((-10,210))
        ax.set_ylim((-1,11))
        ax.set_yticks(range(0,11))
        ss = spk_rec[i,:,:]
        splt.raster(ss, ax, s=1, c="black")

        plt.title("Output Layer - {}".format(targets[i]))
        plt.xlabel("Time step")
        plt.ylabel("Neuron Number")
        fig.tight_layout()
    plt.show()


def print_batch_accuracy(net, data, targets, train=False):
    spk_rec, _, _ = net(data)
    _, data_idx = spk_rec.sum(dim=(1)).max(1)
    _, targets_idx = targets.sum(dim=(1)).max(1)
    acc = np.mean((targets_idx == data_idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer(net, data, test_data, targets, test_targets):
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(net, data, targets, train=True)
    print_batch_accuracy(net, test_data, test_targets, train=False)
    print("\n")

# Architettura e dinamica temporale

In [9]:
# Network Architecture
num_inputs = 28*28
# num_hidden = 1000
num_hidden = 512
num_outputs = 10

# Temporal Dynamics
num_steps = 100
dt = 1e-2
tau_mem = Constant(100e-3)
tau_syn = Constant(50e-3)
threshold = Constant(1.)
bias = Constant(0.)

# Poisson Encode del batch

In [10]:
# - Define a function to encode an input into a poisson event series
def encode_poisson(data: torch.Tensor, num_steps: int, scale: float = 0.1) -> torch.Tensor:
    num_batches, _, _ = data.shape
    data = scale * data.view((num_batches, 1, -1)).repeat((1, num_steps, 1))
    return (torch.rand(data.shape) < (data * scale)).float()

# Encode in spike degli obiettivi

In [11]:
# - Define a function to encode the network target
def encode_class(class_idx: torch.Tensor, num_classes: int, num_steps: int) -> torch.Tensor:
    num_batches = class_idx.numel()
    target = torch.nn.functional.one_hot(class_idx, num_classes = num_classes)
    return target.view((num_batches, 1, -1)).repeat((1, num_steps, 1)).float()

# Network

In [12]:
# Define Network
def DefineNet(num_inputs, num_hidden, num_outputs):
    return Sequential(
        LinearTorch((num_inputs, num_hidden)),
        LIFTorch(
            num_hidden,
            tau_mem=tau_mem,
            tau_syn=tau_syn,
            threshold=threshold,
            bias=bias,
            dt=dt
        ),
        LinearTorch((num_hidden, num_outputs)),
        LIFTorch(
            num_outputs,
            tau_mem=tau_mem,
            tau_syn=tau_syn,
            threshold=threshold,
            bias=bias,
            dt=dt
        )
    )

In [13]:
# Network instantiation
net = DefineNet(num_inputs=num_inputs, num_hidden=num_hidden, num_outputs=num_outputs)
net.to(device=device)

TorchSequential  with shape (784, 10) {
    LinearTorch '0_LinearTorch' with shape (784, 512)
    LIFTorch '1_LIFTorch' with shape (512, 512)
    LinearTorch '2_LinearTorch' with shape (512, 10)
    LIFTorch '3_LIFTorch' with shape (10, 10)
}

In [14]:
run_train = False
state_dict_file_path = "../models/Rockpool-FMNIST-training.pt"

try:
    load_state_dict = torch.load( state_dict_file_path, map_location=device, )
    net.load_state_dict(load_state_dict)
except FileNotFoundError:
    print( "File not found running training" )
    run_train = True

if ( run_train == True ):
    num_epochs = 3
    loss_hist = []
    test_loss_hist = []
    counter = 0
    # loss = nn.CrossEntropyLoss()
    # loss = nn.MSELoss()
    loss = torch.nn.functional.mse_loss
    optimizer = torch.optim.Adam(net.parameters().astorch())#, lr=5e-4)
    net.train()

    # Outer training loop
    for epoch in range(num_epochs):
        # Minibatch training loop
        for data, targets in train_loader:
            print( "Epoch ", epoch, " Iteration: ", counter)
            data = encode_poisson(data.squeeze(), num_steps)

            targets = encode_class(targets, 10, num_steps)

            # - Zero gradients, simulate model
            optimizer.zero_grad()
            # forward pass
            spk_rec, _, _ = net(data.to(device))
            loss_val = loss(spk_rec, targets.to(device))
            print(loss_val)
            # Gradient calculation + weight update
            loss_val.backward()
            optimizer.step()

            # Store loss history for future plotting
            loss_hist.append(loss_val.item())
            counter += 1
            print_batch_accuracy(net, data.to(device), targets.to(device), train=True)

    torch.save( net.state_dict(), state_dict_file_path )

File not found running training
Epoch  0  Iteration:  0
tensor(0.1290, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 18.95%
Epoch  0  Iteration:  1
tensor(0.1186, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 21.88%
Epoch  0  Iteration:  2
tensor(0.1155, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 29.10%
Epoch  0  Iteration:  3
tensor(0.1168, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 30.66%
Epoch  0  Iteration:  4
tensor(0.1170, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 33.98%
Epoch  0  Iteration:  5
tensor(0.1194, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 32.81%
Epoch  0  Iteration:  6
tensor(0.1181, device='cuda:0', grad_fn=<MseLossBackward0>)
Train set accuracy for a single minibatch: 37.50%
Epoch  0  Iteration:  7
tensor

In [15]:
with torch.no_grad():
    net.eval()
    for test_data, test_targets in test_loader:
        test_data = encode_poisson(test_data.squeeze(), num_steps)
        test_data = test_data.to(device)
        test_targets = encode_class(test_targets, 10, num_steps)
        test_targets = test_targets.to(device)
        
        # Test set forward pass
        test_spk, _, _ = net(test_data)

        test_loss = torch.zeros((1), dtype=dtype, device=device)
        test_loss = loss(test_spk, test_targets)

        test_loss_hist.append(test_loss.item())

        # Print train/test loss/accuracy
        print_batch_accuracy(net, test_data.to(device), test_targets.to(device))

Test set accuracy for a single minibatch: 66.80%
Test set accuracy for a single minibatch: 70.31%
Test set accuracy for a single minibatch: 66.80%
Test set accuracy for a single minibatch: 67.38%
Test set accuracy for a single minibatch: 65.04%
Test set accuracy for a single minibatch: 68.75%
Test set accuracy for a single minibatch: 70.31%
Test set accuracy for a single minibatch: 63.48%
Test set accuracy for a single minibatch: 68.55%
Test set accuracy for a single minibatch: 65.04%
Test set accuracy for a single minibatch: 67.97%
Test set accuracy for a single minibatch: 66.99%
Test set accuracy for a single minibatch: 70.12%
Test set accuracy for a single minibatch: 72.85%
Test set accuracy for a single minibatch: 68.55%
Test set accuracy for a single minibatch: 68.95%
Test set accuracy for a single minibatch: 68.55%
Test set accuracy for a single minibatch: 66.41%
Test set accuracy for a single minibatch: 66.60%


In [16]:
print(loss_hist)
print(test_loss_hist)

[0.1290273517370224, 0.1185605525970459, 0.11546093970537186, 0.1167617216706276, 0.11699414253234863, 0.11938867717981339, 0.11809961497783661, 0.11839258670806885, 0.11737891286611557, 0.11790820956230164, 0.1205742210149765, 0.12337695807218552, 0.10972852259874344, 0.11769141256809235, 0.11324024200439453, 0.11206250637769699, 0.11040234565734863, 0.10621485114097595, 0.11032422631978989, 0.10606446117162704, 0.10208985209465027, 0.10369336605072021, 0.10365430265665054, 0.0981757864356041, 0.09608984738588333, 0.10459765791893005, 0.10545703768730164, 0.10166406631469727, 0.09895117580890656, 0.10071679949760437, 0.09396875649690628, 0.09150781482458115, 0.09355664253234863, 0.09354101866483688, 0.09251758456230164, 0.09777539223432541, 0.09299805015325546, 0.09515039622783661, 0.10098633170127869, 0.09423242509365082, 0.0994570329785347, 0.10049609839916229, 0.09323633462190628, 0.09034766256809235, 0.09362109750509262, 0.08873046934604645, 0.09825976938009262, 0.0935976579785347

In [19]:
total = 0
correct = 0

# drop_last switched to False to keep all samples
test_loader = DataLoader(fmnist_test, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad():
  net.eval()
  for data, targets in test_loader:
    data = encode_poisson(data.squeeze(), num_steps)
    targets = encode_class(targets, 10, num_steps)
    targets = targets.to(device)

    # forward pass
    test_spk, _, _ = net(data.to(device))

    # calculate total accuracy
    _, predicted = test_spk.sum(1).max(1)
    total += len(targets.sum(1).max(1).indices)
    correct += (predicted == targets.sum(1).max(1).indices).sum().item()

In [20]:
print(total)
print(correct)
print("{:.2f}".format(100*(correct/total)))

10000
6816
68.16
